# Betweenness Centrality

In this notebook, we will compute the Betweenness centrality of each vertex in our test datase using both cuGraph and NetworkX. The NetworkX and cuGraph processes will be interleaved so that each step can be compared.

Notebook Credits
* Original Authors: Bradley Rees
* Created:   04/24/2019
* Last Edit: 04/24/2020

RAPIDS Versions: 0.14   

Test Hardware

* GV100 32G, CUDA 10.2


## Introduction
Betweenness centrality is a measure of the relative importance of a vertex within the graph based on measuring the number of shortest paths that pass through each vertex.  High betweenness centrality vertices have a greater number of path cross over the vertex.  

See [Betweenness on Wikipedia](https://en.wikipedia.org/wiki/Betweenness_centrality) for more details on the algorithm.



Betweenness centrality of a node 𝑣 is the sum of the fraction of all-pairs shortest paths that pass through 𝑣

<img src="https://latex.codecogs.com/png.latex?c_B(v)&space;=\sum_{s,t&space;\in&space;V}&space;\frac{\sigma(s,&space;t|v)}{\sigma(s,&space;t)}" title="c_B(v) =\sum_{s,t \in V} \frac{\sigma(s, t|v)}{\sigma(s, t)}" />


To compute the Betweenness centrality scores for a graph in cuGraph we use:<br>
__df = cugraph.betweenness_centrality(G)__

    G: cugraph.Graph object
   

Returns:

    df: a cudf.DataFrame object with two columns:
        df['vertex']: The vertex identifier for the vertex
        df['betweenness_centrality']: The betweenness centrality score for the vertex



### _NOTICE_
cuGraph does not currently support the ‘endpoints’ and ‘weight’ parameters as seen in the corresponding networkX call. 

#### cuGraph Notice 
The current version of cuGraph has some limitations:

* Vertex IDs need to be 32-bit integers.
* Vertex IDs are expected to be contiguous integers starting from 0

cuGraph provides the renumber function to mitigate this problem. Input vertex IDs for the renumber function can be of any data type and do not need to be contiguous. The renumber function maps the provided input vertex IDs to 32-bit contiguous integers starting from 0. cuGraph still requires the renumbered vertex IDs to be representable in 32-bit integers. These limitations are being addressed and will be fixed soon.    

### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](../img/zachary_black_lines.png)


The test data has vertex IDs strating at 1.  We will be using the auto-renumber feature of cuGraph to renumber the data so that the starting vertex ID is zero.  The data will be auto-unrenumbered so that the renumbering step is transparent to users. 

### Prep

In [1]:
# Import needed libraries
import cugraph
import cudf
from collections import OrderedDict

In [2]:
# NetworkX libraries
import networkx as nx

### Some Prep

In [3]:
# Define the path to the test data  
datafile='../../datasets/karate-data.csv'

### Read in the data - GPU
cuGraph depends on cuDF for data loading and the initial Dataframe creation

The data file contains an edge list, which represents the connection of a vertex to another.  The `source` to `destination` pairs is in what is known as Coordinate Format (COO).  In this test case, the data is just two columns.  However a third, `weight`, column is also possible

In [4]:
gdf = cudf.read_csv(datafile, delimiter='\t', names=['src', 'dst'], dtype=['int32', 'int32'] )

### Create a Graph 

In [5]:
# create a Graph using the source (src) and destination (dst) vertex pairs from the Dataframe 
G = cugraph.Graph()
G.from_cudf_edgelist(gdf, source='src', destination='dst')

### Call the Betweenness Centrality algorithm

In [6]:
# Call cugraph.betweenness_centrality 
gdf_bc = cugraph.betweenness_centrality(G)

_It was that easy!_  

----

Let's now look at the results

In [7]:
# Find the most important vertex using the scores
# This methods should only be used for small graph
def find_top_scores(_df) :
    m = _df['betweenness_centrality'].max()
    return _df.query('betweenness_centrality >= @m')
        

In [8]:
top_df = find_top_scores(gdf_bc)
top_df

,vertex,betweenness_centrality
0,1,0.437635


In [9]:
# let's sort the data and look at the top 5 vertices
gdf_bc.sort_values(by='betweenness_centrality', ascending=False).head(5)

,vertex,betweenness_centrality
0,1,0.437635
33,34,0.304075
32,33,0.145247
2,3,0.143657
31,32,0.138276


---

## Now compute using NetworkX

In [10]:
# Read the data, this also created a NetworkX Graph 
file = open(datafile, 'rb')
Gnx = nx.read_edgelist(file)

In [11]:
bc_nx = nx.betweenness_centrality(Gnx)

In [12]:
bc_nx_s = sorted(((value, key) for (key,value) in bc_nx.items()), reverse=True)

In [13]:
bc_nx_s[:5]

[(0.4376352813852815, '1'),
 (0.30407497594997596, '34'),
 (0.14524711399711404, '33'),
 (0.14365680615680615, '3'),
 (0.13827561327561327, '32')]

As mentioned, the scores are different but the ranking is the same.

___
Copyright (c) 2019-2020, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___